In [1]:
from functions_untrained_reservoir import *

from reservoirpy.nodes import Reservoir,FORCE,Ridge
from reservoirpy.mat_gen import Initializer
from reservoirpy.observables import spectral_radius

import numpy as np 
import matplotlib.pyplot as plt

### Réseaux de neurones

On cherche ici dans un premier temps à reprdoduire les figures les plus simples de l'article. 

Les mesures de l'article sont effectuées sur 2 réservoirs : 

  - le premier est dit entrainé : on va alors entrainer le réservoir et la sortie
  - le second lui est dit non entrainé : on entraine seulement la sortie
    
 Ce premier script s'intéresse donc à la reproduction de l'entrainement des figures 3, 4 et 5 ainsi qu'aux résultas obtenus sur un réservoir non entrainé.
 
à noter, que la méthode d'entrainement utilisée pour ce script est une méthode dite online utilisant le noeud FORCE de ReservoirPy. Et qu'un autre script traite de la méthode offline pour pouvoir comparer les résultats obtenus.

Données générales 

In [2]:
data = extract_data() #Données d'entrée
transcription = extract_target() #Données d'arrivée 

#Les données suivantes sont utilisés dans les fonctions mais pas dans le script ! 

freq_ent = 6*10**3  #fréquence d'entrée
T_ent = 1/freq_ent  #période d'entrée

freq_out = 10**3 #fréquence de sortie 
T_out = 1/freq_out #période de sortie

ampl_int = 5 #Amplitude d'entrée

Données du réservoir

In [3]:
N = 210 #Nombre de neurones dans le réservoir
M = 12 #Nombres d'entrées
tau = 25*10e-3
dt = 1e-3
lr = dt/tau
sr = 1.8 #spectral radius --> gain ?
g = 1.6 #le gain du réseau ?                             #je comprends pas trop cette donnée. 
connectivity = 0.2 # connectivité entre les neurones (pc dans l'article)
SD = g/np.sqrt(connectivity*N)
mu = 0
I0 = 0.005 #pendant l'entrainement de la sortie            #Non utilisée

Wo = normal(loc=mu,scale = 1/np.sqrt(N))    #Matrice de sortie : Non utilisée


Création de la matrice du réservoir : avec pas d'autapses

In [4]:
W_r.init = Initializer(W_r)

Création de la matrice d'entrée : chaque entrée k est projetée sur le neurone $(k-1)\frac{N}{M} + 1$ jusqu'au neurone $k\frac{N}{M}$ selon une loi normale 

In [5]:
Win = W_in(N,M)

## expérience

Création des données pour l'entrainement

In [6]:
X_input = []
X_target = []
for indS in [1] : 
    for indU in range(1,4):
        for indD in range(10):
            X_input.append(np.transpose(formatage_cocleogram(indS, indU, indD,data)))
            X_target.append(target_xyz(indS, indU, indD,data))

Création du réservoir

In [7]:
reservoir_online, readout_online = Reservoir(units = N,
                                               lr = lr,
                                               sr=sr,
                                               W=W_r.init,
                                               Win=Win,
                                               equation="external",
                                               input_bias=False,
                                               noise_rc = I0), FORCE()
esn_online = reservoir_online >> readout_online


1.8


Entrainement du réservoir

In [8]:
#entrainement du réservoir
for i in range(len(X_input)) : 
    print(i)
    for j in range(1):
        esn_online.train(X_input[i],X_target[i],reset = True)

0
1
2
3
4
5


KeyboardInterrupt: 

Récupération des poids de sortie

In [ ]:
poids_sortie = readout_online.get_param("Wout")
plt.plot(np.transpose(poids_sortie))
print(np.max(poids_sortie))

### Test 1 
Test du réservoir sur des données d'entrainements

###### 1 chiffre

In [ ]:
#Test du réservoir
indS,indU,indD = 1,3,6
X_test = np.transpose(formatage_cocleogram(indS,indU,indD,data))
xyz = esn_online.run(X_test)
    
#récupération des états des neurones
states = etat(reservoir_online,indS,indU,indD,data)

#récupération de l'erreur commise
erreur = error(xyz,indS,indU,indD,data)

Affichage du résultat

In [ ]:
#affichage des résultats
affiche(xyz,indS,indU,indD,data,states,erreur,mode = "online",sr = sr,N=N,I0 = I0)

###### 10 chiffres

In [ ]:
#Test du réservoir sur les 10 chiffres:
liste_chiffre=[]
indS, indU = 1,1
for i in range(10):
    X_test = np.transpose(formatage_cocleogram(indS,indU,i,data))
    liste_chiffre.append(esn_online.run(X_test))

Affichage du résultat

In [ ]:
affiche_chiffre(liste_chiffre,indS,indU,data,mode="online",sr = sr,N=N,I0 = I0)

### Test 2 
On test le réservoir sur des données non entrainées

###### 1 chiffre

In [ ]:
#Test du réservoir
indS,indU,indD = 1,4,6
X_test = np.transpose(formatage_cocleogram(indS,indU,indD,data))
xyz = esn_online.run(X_test)
   
#Récuperation des états des neurones
states = etat(reservoir_online,indS,indU,indD,data)

#Récupération de l'erreur commise (distance euclidienne ici)
erreur = error(xyz,indS,indU,indD,data)

Affichage du chiffre

In [ ]:
affiche(xyz,indS,indU,indD,data,states,erreur,mode = "online",sr = sr,N=N,I0 = I0)

###### 10 chiffres

In [ ]:
#Test du réservoir sur les 10 chiffres:
liste_chiffre=[]
indS, indU = 1,5
for i in range(10):
    X_test = np.transpose(formatage_cocleogram(indS,indU,i,data))
    liste_chiffre.append(esn_online.run(X_test))

Affichage du résultat

In [ ]:
affiche_chiffre(liste_chiffre,indS,indU,data,mode="online",sr = sr,N=N,I0 = I0)

###### Ce qui a bloqué 

Informations pas claires, par exemple on ne sait pas trop quand utiliser le bruit ou non.
Pas de schéma des résultats des réservoirs non entrainés, donc on ne peu pas juger de la cohérence de notre modèle.

# Conclusion

Dans un premier temps nous avons voulu seulement entrainé la sortie de notre réseau. La sortie est entrainé avec une méthode online et avec du bruit (valeur non donnée) est continuellement injecté dans le réservoir pendant l'entrainement.
De plus chaques données étaient entrainées 25 fois (ici je l'ai pas fais sinon mon ordinateur tourne pendant quelques heures).

Nous avons pu remarquer que les résultats étaient peu cohérents avec la target.
De plus, après des tests avec les mêmes valeurs sur un réservoir offline, on remarque que la méthode offline marche "mieux". En effet, avec la méthode online on a un overfitting durant l'apprentissage ce qui nécessite un bruit signifiant dans le réservoir mais qui conduit à des plus grandes perturbations.

Les meilleurs résultats ont été obtenus à en changeant nombres de paramètres : 

   - Méthode d'apprentissage offline
   - N = 210
   - Le bruit a été fixé à 0 
   - sr = 1.0
   - Avec du biais
   - Equation dite "external"
    
Evidemment toutes les combinaisons possibles n'ont pas été essayé.
Les chiffres qu'on pouvait apercevoir étaient le 6 et le 5